In [8]:
import os
import re
import csv
import pandas as pd
from collections import defaultdict


Data saved to files_by_code_color.csv


In [ ]:

# Function to extract code and color from filename
def extract_code_color(filename):
    match = re.match(r'([a-zA-Z0-9]+)_([a-zA-Z0-9]+)_\d+\.\w+', filename)
    if match:
        return match.group(1), match.group(2)
    return None, None

def get_files_by_code_color(directory):
    files_by_code_color = defaultdict(list)

    # List files in directory
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            code, color = extract_code_color(filename)
            if code and color:
                key = (code, color)
                files_by_code_color[key].append(filename)

    return files_by_code_color

def save_to_csv(files_by_code_color, output_csv):
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Code', 'Color'] + ['Filename' + str(i) for i in range(1, max(len(files) for files in files_by_code_color.values()) + 1)])
        for (code, color), files in files_by_code_color.items():
            row = [code, color] + files
            writer.writerow(row)

# Directory containing the files
directory = '/Users/njlalwani/Documents/GitHub/Ecomm-data/photo_scripts/shopify_images'

# Output CSV file
output_csv = 'files_by_code_color.csv'

files_by_code_color = get_files_by_code_color(directory)
save_to_csv(files_by_code_color, output_csv)

print(f'Data saved to {output_csv}')







In [11]:


# Load the uploaded files
files_by_color_path = 'files_by_code_color.csv'
ecom_data_path = '/Users/njlalwani/Library/CloudStorage/OneDrive-Personal/Milano/MILANO_DATA_LOCKED.xlsx'

files_by_color = pd.read_csv(files_by_color_path)
ecom_data = pd.read_excel(ecom_data_path, sheet_name=None)

# Load the MAIN DATASHEET from the Excel file
main_datasheet = ecom_data['MAIN DATASHEET']

# Extracting relevant columns from both dataframes
main_datasheet_relevant = main_datasheet[['STYLE NO.', 'COLOR']].drop_duplicates()
files_by_color_relevant = files_by_color[['Code', 'Color']].drop_duplicates()

# Renaming columns for consistency before merging
main_datasheet_relevant.columns = ['Style', 'Color']
files_by_color_relevant.columns = ['Style', 'Color']

# Finding combinations in main_datasheet that are not in files_by_color
absent_combinations = pd.merge(main_datasheet_relevant, files_by_color_relevant, on=['Style', 'Color'], how='left', indicator=True)
absent_combinations = absent_combinations[absent_combinations['_merge'] == 'left_only'].drop(columns=['_merge'])

# Clean the SIZE_US column by converting it to numeric, errors='coerce' will turn non-numeric values into NaN
main_datasheet['SIZE_US'] = pd.to_numeric(main_datasheet['SIZE_US'], errors='coerce')

# Filter main datasheet for SIZE_US in the range 4 to 14
filtered_main_datasheet = main_datasheet[main_datasheet['SIZE_US'].between(4, 14)]

# Merge filtered main datasheet with absent combinations to find the relevant inventory
relevant_inventory = pd.merge(filtered_main_datasheet, absent_combinations, left_on=['STYLE NO.', 'COLOR'], right_on=['Style', 'Color'])

# Select the relevant columns
relevant_inventory = relevant_inventory[['STYLE NO.', 'COLOR', 'BOX NUMBER', 'SIZE_US']]

# Save the result to a CSV file
output_path = 'inventory_of_absent_combinations.csv'
relevant_inventory.to_csv(output_path, index=False)

# Display the path to the saved file
print(f'Results saved to {output_path}')


Results saved to inventory_of_absent_combinations.csv


In [13]:
import pandas as pd

# Load the CSV file
file_path = 'inventory_of_absent_combinations.csv'
df = pd.read_csv(file_path)

def find_best_boxes(df):
    results = []

    # Get unique style-color combinations
    style_color_combinations = df[['STYLE NO.', 'COLOR']].drop_duplicates()

    for _, row in style_color_combinations.iterrows():
        style = row['STYLE NO.']
        color = row['COLOR']

        # Filter the dataframe for the current style-color combination
        subset = df[(df['STYLE NO.'] == style) & (df['COLOR'] == color)]

        # Find the optimal set of boxes to cover sizes 4-14
        sizes_needed = set(range(4, 15))
        boxes_needed = set()
        
        while sizes_needed:
            # Find the box that covers the most uncovered sizes
            box_counts = subset[subset['SIZE_US'].isin(sizes_needed)]['BOX NUMBER'].value_counts()
            if box_counts.empty:
                # If no boxes cover the remaining sizes, break the loop
                break
            best_box = box_counts.idxmax()
            boxes_needed.add(best_box)

            # Remove the sizes covered by the selected box
            sizes_covered = subset[subset['BOX NUMBER'] == best_box]['SIZE_US']
            sizes_needed -= set(sizes_covered)

        # Store the result for the current style-color combination
        results.append({'STYLE NO.': style, 'COLOR': color, 'BOXES NEEDED': list(boxes_needed)})

    return pd.DataFrame(results)

# Apply the function to the dataframe
result_df = find_best_boxes(df)

# Save the results to a new CSV file
output_file_path = 'optimal_boxes_for_style_color_combinations.csv'
result_df.to_csv(output_file_path, index=False)

# Display the saved file path to the user
output_file_path


'optimal_boxes_for_style_color_combinations.csv'

In [15]:
import pandas as pd

# Load the CSV and Excel files
files_by_code_color_path = 'files_by_code_color.csv'
ecomm_data_path = '/Users/njlalwani/Library/CloudStorage/OneDrive-Personal/Milano/MILANO_DATA_LOCKED.xlsx'

files_by_code_color_df = pd.read_csv(files_by_code_color_path)
ecomm_data_df = pd.read_excel(ecomm_data_path)

# Extract (style, color) combinations from files_by_code_color
files_by_code_color_combinations = files_by_code_color_df[['Code', 'Color']].drop_duplicates()

# Extract (style, color) combinations from ecomm_data
ecomm_data_combinations = ecomm_data_df[['STYLE NO.', 'COLOR']].drop_duplicates()

# Rename columns for consistent comparison
files_by_code_color_combinations.columns = ['STYLE NO.', 'COLOR']

# Find combinations present in ecomm_data but not in files_by_code_color
unique_combinations = pd.merge(ecomm_data_combinations, files_by_code_color_combinations, on=['STYLE NO.', 'COLOR'], how='left', indicator=True)
unique_combinations = unique_combinations[unique_combinations['_merge'] == 'left_only'].drop(columns=['_merge'])

# Save the unique combinations to a CSV file
unique_combinations_path = 'unique_style_color_combinations.csv'
unique_combinations.to_csv(unique_combinations_path, index=False)

# Display the saved file path
print(unique_combinations_path)


unique_style_color_combinations.csv
